In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold,train_test_split
import gc
import datetime
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)

In [ ]:
policy_data = pd.read_csv("/kaggle/input/policy_data.csv")
payment_history = pd.read_csv("/kaggle/input/payment_history.csv")
client_data = pd.read_csv("/kaggle/input/client_data.csv")
sample_sub = pd.read_csv("/kaggle/input/sample_sub.csv")
train = pd.read_csv("/kaggle/input/train.csv")

In [ ]:
policy_data

In [ ]:
policy_data['NPR_PREMIUM']=policy_data['NPR_PREMIUM'].fillna(policy_data['NPR_PREMIUM'].mean())
policy_data['NPR_SUMASSURED']=policy_data['NPR_SUMASSURED'].fillna(policy_data['NPR_SUMASSURED'].mean())
policy_data['NLO_AMOUNT']=policy_data['NLO_AMOUNT'].fillna(policy_data['NLO_AMOUNT'].mean())

In [ ]:
payment_history

In [ ]:
payment_history['AMOUNTPAID']=payment_history['AMOUNTPAID'].fillna(payment_history['AMOUNTPAID'].mean())

In [ ]:
client_data

In [ ]:
client_data['NAD_ADDRESS1']=client_data['NAD_ADDRESS1'].fillna(method='bfill')
client_data['NAD_ADDRESS2']=client_data['NAD_ADDRESS2'].fillna(method='bfill')
client_data['NPH_LASTNAME']=client_data['NPH_LASTNAME'].fillna(method='bfill')

In [ ]:
sample_sub

In [ ]:
train

In [ ]:
policy_data=policy_data.drop_duplicates( "Policy ID" , keep='first')

In [ ]:
train_df = pd.merge(left=train, right=policy_data, how='left', left_on='Policy ID', right_on='Policy ID')

In [ ]:
payment_history=payment_history.drop_duplicates( "Policy ID" , keep='first')

In [ ]:
train_df = pd.merge(left=train_df, right=payment_history, how='left', left_on='Policy ID', right_on='Policy ID')

In [ ]:
client_data=client_data.drop_duplicates( "Policy ID" , keep='first')

In [ ]:
train_df = pd.merge(left=train_df, right=client_data, how='left', left_on='Policy ID', right_on='Policy ID')

In [ ]:
k=['DATEPAID','POSTDATE','PREMIUMDUEDATE','NPH_TITLE','NPH_LASTNAME_y','NPH_SEX','NPH_BIRTHDATE','NAD_ADDRESS1','NAD_ADDRESS2']
for i in k:
    train_df[i]=train_df[i].fillna(method='bfill')
train_df['AMOUNTPAID']=train_df['AMOUNTPAID'].fillna(train_df['AMOUNTPAID'].mean())

In [ ]:
c=[]
for i in train_df['Lapse Year']:
    if i=='?':
        c.append(2020)
    else:
        c.append(i)
train_df['Lapse Year']=c

In [ ]:
k=['NP2_EFFECTDATE','DATEPAID','POSTDATE','PREMIUMDUEDATE']
for i in k:
  train_df[i] = pd.to_datetime(train_df[i])

In [ ]:
k=['NP2_EFFECTDATE','DATEPAID','POSTDATE','PREMIUMDUEDATE']
for i in k:
  train_df[i+'_'+'year'] = train_df[i].dt.year
  train_df[i+'_'+'day'] = train_df[i].dt.day
  train_df[i+'_'+'weekofyear'] = train_df[i].dt.weekofyear
  train_df[i+'_'+'month'] = train_df[i].dt.month
  train_df[i+'_'+'dayofweek'] = train_df[i].dt.dayofweek
  train_df[i+'_'+'weekend'] = (train_df[i].dt.weekday >=5).astype(int)
  train_df[i+'_'+'hour'] = train_df[i].dt.hour
  train_df[i+'_'+'minute'] = train_df[i].dt.minute

In [ ]:
k=['NP2_EFFECTDATE','DATEPAID','POSTDATE','PREMIUMDUEDATE']
for i in k:
    del train_df[i]

In [ ]:
train_df['NP2_EFFECTDATE_year'].value_counts()

In [ ]:
train=train_df.loc[train_df['NP2_EFFECTDATE_year'] != 2020]
test=train_df.loc[train_df['NP2_EFFECTDATE_year'] == 2020]

In [ ]:
test_df=train_df.loc[train_df['Lapse'] != '1']

In [ ]:
test_df

In [ ]:
del test_df['Lapse']

In [ ]:
c=['PPR_PRODCD','NPH_LASTNAME_x','NLO_TYPE','NPH_TITLE','NPH_LASTNAME_y','NPH_SEX','NPH_BIRTHDATE','NAD_ADDRESS1','NAD_ADDRESS2','AAG_AGCODE','PCL_LOCATCODE','OCCUPATION','CATEGORY']
for i in c:
  loc = np.append(train[i].values, test[i].values, axis=0)
  from sklearn.preprocessing import LabelEncoder
  l=LabelEncoder()
  l.fit(list(set(loc)))
  train[i]=l.transform(train[i])
  test_df[i]=l.transform(test_df[i])

In [ ]:
train['Lapse Year']=train['Lapse Year'].astype(int)
test_df['Lapse Year']=test_df['Lapse Year'].astype(int)

In [ ]:
c=test['Policy ID']
del train['Policy ID']
del test_df['Policy ID']

In [ ]:
d=train['Lapse']
del train['Lapse']
train['Lapse']=d

In [ ]:
m=[]
for i in train['Lapse']:
    if i=='?':
        m.append(1)
    else:
        m.append(0)
train['Lapse']=m

# model

In [ ]:
train_df=train

In [ ]:
X = train_df.drop(labels=['Lapse'], axis=1)
y = train_df['Lapse'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [ ]:

X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'multiclass',
         'num_class': 2,
         'boosting': 'gbdt',  
         'metric': 'multi_logloss',
         'learning_rate': 0.01, 
         'num_iterations': 100,
         'num_leaves': 100,
         'max_depth': -1,
         'min_data_in_leaf': 15,
         'bagging_fraction':0.9,
         'bagging_freq': 1,
         'feature_fraction': 0.7,
         'lambda_l2': 0.8,
         'min_data_per_group': 75,
         'max_bin': 255,
         'is_unbalance':False
         }

clf = lgb.train(params=param, 
                early_stopping_rounds=200,
                verbose_eval=100,
                train_set=train_data,
                valid_sets=[test_data])

y_pred = clf.predict(X_cv)

In [ ]:
log_loss(y_cv, y_pred)

In [ ]:
import seaborn as sns
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(), X.columns), reverse=True)[:], columns=['Value','Feature'])
plt.figure(figsize=(10,10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()

In [ ]:
y_pred = clf.predict(test_df)

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_sub.columns)
submission.head()

In [ ]:
submission = pd.DataFrame({

        "Policy ID": sample_sub['Policy ID'],
        "Lapse": submission['Policy ID']
    })
submission.to_csv('./dartf.csv', index=False)
print(submission)